In [1]:
import dhg
import torch
import optuna
import scipy.sparse as sp
from itertools import combinations
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
import pickle
from copy import deepcopy
import torch.optim as optim
from dhg import Hypergraph
from dhg.data import *
from dhg.random import set_seed
import dhg
from dhg.metrics import HypergraphVertexClassificationEvaluator as Evaluator
from dhg.structure.hypergraphs import Hypergraph
import math
import time
import gc
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from scipy.sparse import csr_matrix
from tqdm import tqdm
from sklearn.model_selection import train_test_split


/users/Min/miniconda/envs/hy/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load data
with open('data/graphs/contact-high-school/hyperedges-contact-high-school.txt', 'r') as file:
    edge_list = [tuple(map(lambda x: int(x) - 1, line.strip().split(','))) for line in file]

num_vertices = len(set(item for sublist in edge_list for item in sublist))

# print(len(edge_list), num_vertices)
hg = Hypergraph(num_vertices, edge_list)
print(hg)

labels = []
with open ('data/graphs/contact-high-school/node-labels-contact-high-school.txt', 'r') as file:
    for line in file:
        labels.append(int(line)-1)
labels = torch.LongTensor(labels)

# 设置随机种子，以确保结果可复现
random_seed = 42

node_idx = [i for i in range(num_vertices)]
# 将idx_test划分为训练（50%）、验证（25%）和测试（25%）集
idx_train, idx_temp = train_test_split(node_idx, test_size=0.1, random_state=random_seed)
idx_val, idx_test = train_test_split(idx_temp, test_size=0.5, random_state=random_seed)

# 确保划分后的集合没有重叠
assert len(set(idx_train) & set(idx_val)) == 0
assert len(set(idx_train) & set(idx_test)) == 0
assert len(set(idx_val) & set(idx_test)) == 0

train_nid = torch.LongTensor(idx_train)
val_nid = torch.LongTensor(idx_val)
test_nid = torch.LongTensor(idx_test)


train_mask = torch.zeros(num_vertices, dtype=torch.bool)
val_mask = torch.zeros(num_vertices, dtype=torch.bool)
test_mask = torch.zeros(num_vertices, dtype=torch.bool)
train_mask[train_nid] = True
val_mask[val_nid] = True
test_mask[test_nid] = True

features = torch.eye(num_vertices)
# adj_matrix = adjacency_matrix(hg, s=1, weight=False)

data = {}
data['num_vertices'] = len(node_idx)
data['edge_list'] = edge_list
data['labels'] = labels
data['train_mask'] = train_mask
data['val_mask'] = val_mask
data['test_mask'] = test_mask

Hypergraph(num_v=327, num_e=7818)


In [4]:
adj_matrix = hg.H.to_dense()


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [5]:
adj_matrix.shape,hg.H.shape

(torch.Size([327, 7818]), torch.Size([327, 7818]))

In [6]:
hg.H

tensor(indices=tensor([[   0,    0,    0,  ...,  326,  326,  326],
                       [  58,  126,  289,  ..., 4438, 4720, 6167]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(327, 7818), nnz=18192, layout=torch.sparse_coo)

In [13]:
hg.D_e

tensor(indices=tensor([[   0,    1,    2,  ..., 7815, 7816, 7817],
                       [   0,    1,    2,  ..., 7815, 7816, 7817]]),
       values=tensor([3., 4., 2.,  ..., 2., 2., 2.]),
       size=(7818, 7818), nnz=7818, layout=torch.sparse_coo)

In [21]:
adj_matrix = hg.H
# adj_matrix = adj_matrix.coalesce()  # Ensure that indices are unique
# adj_matrix = sp.coo_matrix((adj_matrix.values(), (adj_matrix.indices()[0], adj_matrix.indices()[1])), shape=adj_matrix.shape)

# Compute node degrees
node_degrees = torch.sparse.sum(hg.H, dim=1).to_dense()

# Compute hyperedge degrees
hyperedge_degrees = torch.sparse.sum(hg.H, dim=0).to_dense()

# 计算节点的度矩阵的逆平方根
node_degrees_inv_sqrt = 1.0 / torch.sqrt(node_degrees)

# 计算超边的度矩阵的逆平方根
hyperedge_degrees_inv_sqrt = 1.0 / torch.sqrt(hyperedge_degrees)

# 创建对角矩阵，其对角线元素为节点度矩阵的逆平方根
node_degrees_inv_sqrt_diag = torch.diag(node_degrees_inv_sqrt)

# 创建对角矩阵，其对角线元素为超边度矩阵的逆平方根
hyperedge_degrees_inv_sqrt_diag = torch.diag(hyperedge_degrees_inv_sqrt)

print(node_degrees_inv_sqrt_diag.shape, adj_matrix.shape, hyperedge_degrees_inv_sqrt_diag.shape)

# 归一化节点和超边的关联矩阵
normalized_relation_matrix = node_degrees_inv_sqrt_diag @ adj_matrix @ hyperedge_degrees_inv_sqrt_diag


torch.Size([327, 327]) torch.Size([327, 7818]) torch.Size([7818, 7818])


NotImplementedError: Could not run 'aten::sparse_dim' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::sparse_dim' is only available for these backends: [SparseCPU, SparseCUDA, SparseMeta, SparseCsrCPU, SparseCsrCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMeta, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDy